<a href="https://colab.research.google.com/github/Anshuman-37/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IDEA- 
0. Downloading Data and Reading it


> Use KAGGLE JSON file to directly donwload data in colab and use it if having any problem search on google how to download kaggle data.

> ~~Then read the data in pyspark dataframe~~

Done


1. Change Data Type 

> ~~Everything is a string type first convert it to the required type and change the schema~~ 

~~<em>Answer -  I think I will brute force on this one getting all the column names and changing the schema</em>~~

Done

2. Data visualization


> ~~We need to visualize the data and check for imbalance. To do that we need to concatinate the data~~ 

~~Answer - Concatinate Biden and Trumps data in a single data frame~~ 

Done

3. Data cleaning

> ~~Haven't thought about it in detail but we need to do normal nltk steps to do this~~

~~Answer - Used Regular Expression to remove the useless characters in the string~~

Done 

4. Analysis

> Think of using a library or we need to do it on our own ? 

~~Answer - Create a new column and apply text blob on it to get the result.~~

> Do we need to use vector assembler and the other shit as well to do this. I am not sure..
we need to apply some ML algorithm i was thinking in k-means (something like this: https://medium.com/analytics-vidhya/congressional-tweets-using-sentiment-analysis-to-cluster-members-of-congress-in-pyspark-10afa4d1556e ) but we can do another one.












## Downloading the data from Kaggle

In [1]:
## Prerequisites
!pip install kaggle --upgrade
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
## Downloading data command
!kaggle datasets download -d manchunhui/us-election-2020-tweets

us-election-2020-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!mkdir content/data
!unzip us-election-2020-tweets.zip -d data

mkdir: cannot create directory ‘content/data’: No such file or directory
Archive:  us-election-2020-tweets.zip
replace data/hashtag_donaldtrump.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


## Download Pyspark

In [4]:
!pip install pyspark 

## Import Header Files

In [5]:
## Python 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import re


# stopwords, tokenizer, stemmer
import nltk  
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist

# spell correction, lemmatization
from textblob import TextBlob
from textblob import Word

## Spark
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType , FloatType, IntegerType, ByteType
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import udf

# Create Spark Session
spark = SparkSession.builder.master("local[1]").appName('Twitter Sentiment Analysis').getOrCreate()

## Reading the Data

#### Defining the Schema

In [6]:

# Created the schema that we need 
# Fucking Brute Force I dont like it
mySchema = StructType([ StructField('created_at', StringType(), True)\
                       ,StructField('tweet_id', FloatType(), True)\
                       ,StructField('tweet',StringType(),True)\
                       ,StructField('likes',FloatType(),True)\
                       ,StructField('retweet_count',FloatType(),True)\
                       ,StructField('source',StringType(),True)\
                       ,StructField('user_id',FloatType(),True)\
                       ,StructField('user_name',StringType(),True)\
                       ,StructField('user_screen_name',StringType(),True)\
                       ,StructField('user_description',StringType(),True)\
                       ,StructField('user_join_date',StringType(),True)\
                       ,StructField('user_followers_count',FloatType(),True)\
                       ,StructField('user_location',StringType(),True)\
                       ,StructField('lat',FloatType(),True)\
                       ,StructField('long',FloatType(),True)\
                       ,StructField('city',StringType(),True)\
                       ,StructField('country',StringType(),True)\
                       ,StructField('continent',StringType(),True)\
                       ,StructField('state',StringType(),True)\
                       ,StructField('state_code',StringType(),True)\
                       ,StructField('collected_at',StringType(),True)\
                       ])

In [7]:
trump_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option("quote", "\"")\
                     .option("escape", "\"")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_donaldtrump.csv')
trump_df.cache()
#trump_df.unpersist()
print('Trump DataFrame Schema')

trump_df.printSchema()
display(trump_df)

biden_df = spark.read.option('multiline',True)\
                     .option('header', 'true')\
                     .option('lineterminator',"\n")\
                     .option("quote", "\"")\
                     .option("escape", "\"")\
                     .schema(mySchema)\
                     .csv('/content/data/hashtag_joebiden.csv')

biden_df.cache()
print('Biden DataFrame Schema')
biden_df.printSchema()
display(biden_df)

Trump DataFrame Schema
root
 |-- created_at: string (nullable = true)
 |-- tweet_id: float (nullable = true)
 |-- tweet: string (nullable = true)
 |-- likes: float (nullable = true)
 |-- retweet_count: float (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: float (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: float (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- long: float (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)



DataFrame[created_at: string, tweet_id: float, tweet: string, likes: float, retweet_count: float, source: string, user_id: float, user_name: string, user_screen_name: string, user_description: string, user_join_date: string, user_followers_count: float, user_location: string, lat: float, long: float, city: string, country: string, continent: string, state: string, state_code: string, collected_at: string]

Biden DataFrame Schema
root
 |-- created_at: string (nullable = true)
 |-- tweet_id: float (nullable = true)
 |-- tweet: string (nullable = true)
 |-- likes: float (nullable = true)
 |-- retweet_count: float (nullable = true)
 |-- source: string (nullable = true)
 |-- user_id: float (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_join_date: string (nullable = true)
 |-- user_followers_count: float (nullable = true)
 |-- user_location: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- long: float (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- collected_at: string (nullable = true)



DataFrame[created_at: string, tweet_id: float, tweet: string, likes: float, retweet_count: float, source: string, user_id: float, user_name: string, user_screen_name: string, user_description: string, user_join_date: string, user_followers_count: float, user_location: string, lat: float, long: float, city: string, country: string, continent: string, state: string, state_code: string, collected_at: string]

#### Trump Data Stats

In [8]:
# Took me 35 seconds to run
trump_df.show(20)

+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-----------------+--------------------+-------------+--------------------+----------+--------------------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|             source|      user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|             city|             country|    continent|               state|state_code|        collected_at|
+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-----------------+---------

In [9]:
# Took me 2 minutes to run one command carefull while running
trump_df.summary().show()

+-------+-------------------+--------------------+--------------------+------------------+------------------+--------------+--------------------+--------------------+-------------------+--------------------+-------------------+--------------------+-------------+------------------+------------------+--------------------+-----------+-------------+----------------+----------+--------------------+
|summary|         created_at|            tweet_id|               tweet|             likes|     retweet_count|        source|             user_id|           user_name|   user_screen_name|    user_description|     user_join_date|user_followers_count|user_location|               lat|              long|                city|    country|    continent|           state|state_code|        collected_at|
+-------+-------------------+--------------------+--------------------+------------------+------------------+--------------+--------------------+--------------------+-------------------+--------------------

#### Biden Data Stats

In [10]:
biden_df.show(5)

+-------------------+-------------+--------------------+-----+-------------+------------------+-------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+----+--------------------+-------------+----------+----------+--------------------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|            source|      user_id|         user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|      long|city|             country|    continent|     state|state_code|        collected_at|
+-------------------+-------------+--------------------+-----+-------------+------------------+-------------+------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+----+--------------------+-------------+----------+----------+--------------------+
|

In [11]:
# Took me 1 minute 21 seconds to run this command carefull while running
biden_df.summary().show()

+-------+-------------------+--------------------+--------------------+------------------+-----------------+--------------------+--------------------+-------------+--------------------+--------------------+-------------------+--------------------+-------------+------------------+------------------+------------+-----------+-------------+----------------+----------+--------------------+
|summary|         created_at|            tweet_id|               tweet|             likes|    retweet_count|              source|             user_id|    user_name|    user_screen_name|    user_description|     user_join_date|user_followers_count|user_location|               lat|              long|        city|    country|    continent|           state|state_code|        collected_at|
+-------+-------------------+--------------------+--------------------+------------------+-----------------+--------------------+--------------------+-------------+--------------------+--------------------+------------------

#### Concatinating Data 

In [12]:
## Adding a column to the dataframe 
trump_df = trump_df.withColumn("Candidate", lit('TRUMP'))
trump_df.show()

+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-----------------+--------------------+-------------+--------------------+----------+--------------------+---------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|             source|      user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|             city|             country|    continent|               state|state_code|        collected_at|Candidate|
+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-------

In [13]:
biden_df = biden_df.withColumn("Candidate", lit('BIDEN'))
biden_df.show()

+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+-------+--------------------+-------------+----------+----------+--------------------+---------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|             source|      user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|      long|   city|             country|    continent|     state|state_code|        collected_at|Candidate|
+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+----------+-------+--------------------+-------------+-------

In [14]:
data = trump_df.union(biden_df)
data.cache()
data.show()

+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-----------------+--------------------+-------------+--------------------+----------+--------------------+---------+
|         created_at|     tweet_id|               tweet|likes|retweet_count|             source|      user_id|           user_name|user_screen_name|    user_description|     user_join_date|user_followers_count|       user_location|      lat|       long|             city|             country|    continent|               state|state_code|        collected_at|Candidate|
+-------------------+-------------+--------------------+-----+-------------+-------------------+-------------+--------------------+----------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+-------

## EDA



```

```

In [15]:
## Checking the data imbalance
data.groupBy('Candidate').count().withColumnRenamed("count","Number of Data").show()

+---------+--------------+
|Candidate|Number of Data|
+---------+--------------+
|    TRUMP|        970919|
|    BIDEN|        776886|
+---------+--------------+



In [16]:
## Comparrsion for the number of tweets
data.groupby('Candidate').agg(F.expr('count(tweet)')\
                              .alias('Number of Tweets')).show()

+---------+----------------+
|Candidate|Number of Tweets|
+---------+----------------+
|    TRUMP|          970919|
|    BIDEN|          776886|
+---------+----------------+



In [17]:
# comparison of likes
data.groupby('Candidate').agg(F.expr('count(likes)')\
                              .alias('Number of Likes')).show()


+---------+---------------+
|Candidate|Number of Likes|
+---------+---------------+
|    TRUMP|         970919|
|    BIDEN|         776886|
+---------+---------------+



In [18]:
# Can someone please remove the null that is being shown also show only top 10 rows 
data.groupby('Country').count()\
.withColumnRenamed('count','Number of Data')\
.dropna()\
.sort(F.desc('Number of Data')).show()


+--------------------+--------------+
|             Country|Number of Data|
+--------------------+--------------+
|United States of ...|        332495|
|       United States|         61905|
|      United Kingdom|         58051|
|               India|         40091|
|             Germany|         35379|
|              France|         35299|
|              Canada|         27805|
|               Italy|         20076|
|           Australia|         14899|
|              Mexico|         10903|
|              Turkey|         10368|
|     The Netherlands|          9587|
|              Brazil|          8735|
|            Pakistan|          8597|
|               Spain|          7252|
|             Ireland|          5452|
|         Netherlands|          5279|
|            Colombia|          4425|
|           Argentina|          4404|
|           Venezuela|          4333|
+--------------------+--------------+
only showing top 20 rows



In [19]:
# Making USA to US 
data = data.withColumn('Country', regexp_replace('Country', 'United States of America', 'US'))
data = data.withColumn('Country', regexp_replace('Country', 'United States', 'US'))

## Make sure by checking the count of the data 
data.groupby('Country').count()\
.withColumnRenamed('count','Number of Data')\
.dropna()\
.sort(F.desc('Number of Data')).show()

+---------------+--------------+
|        Country|Number of Data|
+---------------+--------------+
|             US|        394400|
| United Kingdom|         58051|
|          India|         40091|
|        Germany|         35379|
|         France|         35299|
|         Canada|         27805|
|          Italy|         20076|
|      Australia|         14899|
|         Mexico|         10903|
|         Turkey|         10368|
|The Netherlands|          9587|
|         Brazil|          8735|
|       Pakistan|          8597|
|          Spain|          7252|
|        Ireland|          5452|
|    Netherlands|          5279|
|       Colombia|          4425|
|      Argentina|          4404|
|      Venezuela|          4333|
|          Chile|          3844|
+---------------+--------------+
only showing top 20 rows



## Preprocessing the Data(Tweets)

In [20]:
!pip install nltk
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_tweets(text, stem=False, lemmatize=False):
    ## If statement for fault tolerance
    if isinstance(text,str):
        text = text.lower()
        # Remove links, special characters , numbers punctuation etc. 
        chars_and_links= r'\d+|http?\S+|[^A-Za-z0-9]+'
        # Cleaned the text
        text = re.sub(chars_and_links,' ',text)

        filtered_text = []
        # Tokenize the text
        words = word_tokenize(text) 
        # Remove stopwords and stem
        for word in words:
            # Words in stop words or not
            if not word in stop_words:
                if stem:
                    filtered_text.append(ps.stem(word))
                elif lemmatize:
                    filtered_text.append(Word(word).lemmatize())
                else:
                    filtered_text.append(word)
        # Return the filtered text
        return ' '.join(filtered_text)
    else:
        return text

In [22]:
Preprocess_UDF = udf(lambda x: preprocess_tweets(x))  

tokenized_df = data.withColumn('tokenized_tweet', Preprocess_UDF(F.col('tweet')))

In [ ]:
def classify(polarity):
    ''' This is UDF function for classification '''
    if polarity > 0:
        return 'positive'
    if polarity == 0:
        return 'neutral'
    return 'negative'

## UDF to calculate polarity
get_polarity = udf(lambda text: TextBlob(text).sentiment.polarity)
## UDF to calcularte the subjectivity
get_subjectivity = udf(lambda text: TextBlob(text).sentiment.subjectivity)
## UDF for getting classification 
classify_sentiment = udf(classify)

## Using text blob to get the polarity 
tokenized_df = tokenized_df.withColumn('polarity', get_polarity(tokenized_df.tokenized_tweet))\
                            .withColumn('subjectivity', get_subjectivity(tokenized_df.tokenized_tweet))

sent_analysis = tokenized_df.withColumn('classification', classify_sentiment(tokenized_df.polarity))
                                
sent_analysis.show()
sent_analysis.count()                  

## Droping useless attributes from the data frame

In [24]:
# Droping useless attributes from the main data frame
df = sent_analysis.drop("tweet_id","created_at","retweet_count","source", "user_id", "user_name", "user_screen_name","user_description", "user_join_date", "user_followers_count","user_location", "lat", "long", "city", "state_code", "collected_at","continent", "polarity", "subjectivity","tokenized_tweet")

In [25]:
# Printing the modified dataframe
#df.show()
df.write.csv('Tweet_Analysis.csv')
## Converting to pandas data frame for simplicity
# import pandas as pd 
# pandas_df = df.toPandas()
# print(pandas_df)

AnalysisException: ignored

In [ ]:
# import pandas as pd 
# pandas_df = pd.read_csv('/content/Tweet_Analysis.csv/part-00001-b60952d2-6944-4377-84ec-ec7b62117f35-c000.csv')
# pandas_df.show()
import pandas as pd 
pandas_df = df.toPandas()
#print(pandas_df)

In [ ]:
print(pandas_df)

In [ ]:
pandas_df = pandas_df.groupby(['country','Candidate'])['tweet'].count().reset_index()


In [ ]:
## Creating a temporary table on Data Frame
# df.createOrReplaceTempView("Tweet_Analysis")
# # Checking the Sql query
# df1 = spark.sql("SELECT * from Tweet_Analysis")
# df1.show()

In [ ]:
## Candidate group by 
#df2 = spark.sql("SELECT Country from Tweet_Analysis group by Candidate ")
#df2 = spark.sql("SELECT Country from Tweet_Analysis group by Candidate ")

# df2 = spark.sql("SELECT * from Tweet_Analysis")
# df2.show()
#.withColumnRenamed('count','Number of Data')\
#.dropna()\

### Dalila's Try

In [ ]:
@udf(returnType=ByteType())
def classify(polarity):
    if polarity > 0:
      return 1
    if polarity == 0:
      return 0
    return -1

@udf(returnType=FloatType())
def get_polarity(text):
    return TextBlob(text).sentiment.polarity

@udf(returnType=FloatType())
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

tokenized_df = tokenized_df.withColumn('polarity', get_polarity(tokenized_df.tokenized_tweet))\
                            .withColumn('subjectivity', get_subjectivity(tokenized_df.tokenized_tweet))

sent_analysis = tokenized_df.withColumn('classification', classify(tokenized_df.polarity))
                                
sent_analysis.show()




In [ ]:
sent_analysis.write.parquet("tweetscandidate1.parquet").save()

In [ ]:
DF=spark.read.parquet("tweetscandidate.parquet")

In [ ]:
sent_analysis.write.csv('tweetscandidate.csv')

In [ ]:

parDF=spark.read.parquet("tweetsclean.parquet")

In [ ]:
sent_analysis.write.csv('tweets1.csv')

In [ ]:
parDF.show()
parDF.count()

In [ ]:
df = sent_analysis.drop("created_at","retweet_count","source", "user_id", "user_name", "user_screen_name","user_description", "user_join_date", "user_followers_count","user_location", "lat", "long", "city", "continent", "state", "state_code", "collected_at")

In [ ]:
df.show()

In [ ]:
sent_analysis.groupBy('classification').count().withColumnRenamed("count","Number of Data").show()


In [ ]:
train, test = sent_analysis.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.regression import GBTRegressor

vectorAssembler = VectorAssembler(inputCols=['polarity','subjectivity'],
                                  outputCol='features')

# Takes the "features" column and learns to predict "classification"
gbt = GBTRegressor(labelCol="classification")


In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [2, 5])\
  .addGrid(gbt.maxIter, [10, 20])\
  .build()

from pyspark.ml.evaluation import RegressionEvaluator

# We define an evaluation metric.  This tells CrossValidator how well we are doing by comparing the true labels with predictions.
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
# Declare the CrossValidator, which runs model tuning for us.
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, cv])

In [ ]:

pipelineModel = pipeline.fit(train)

In [ ]:
predictions = pipelineModel.transform(test)

In [ ]:
#predictions.select